# Does compositional data affect differential abundance calculations?

Start by generating two populations, where samples in each population represent a vector of 1000 microbial species abundances. 

Importantly, the two populations are both normalized such that the sum of each vector equals a constant. This means that the data are compositional, and each element represents a proportion of a whole. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind as tt
from scipy.stats import gamma, poisson

In [2]:
# define properties of simulation 

# number of taxa 
n = 1000 
# define number of samples from population 1 
m1 = 20 
# define number of samples from population 2 
m2 = 30 

# define "p" percent of taxa to be differentially abundant 
p = .2

In [3]:
# choose taxa such that 10% have high abundance, 30% have medium abundance, and 60% low abundance 
inds = np.array([0]*int(.6*n) + [1]*int(.3*n) + [2]*int(.1*n))

# define a nominal vector from population 1 
a1 = 50*np.ones(n)   # low abundance 
a1[inds==1] = 200    # medium abundance 
a1[inds==2] = 10000  # High abundance  

# need to make "p" percent of taxa differentially abundant 
# we expect p*n taxa to be significantly different across population 1 and population 2 
dff_inds = np.array([0]*int((1-p)*n) + [1]*int(p*n))
np.random.shuffle(dff_inds)
for i, d in enumerate(dff_inds): # for each taxa 
    if d:                        # if the taxa should be differentially abundant 
        # randomly switch ind 
        all_ind_options = np.array([0, 1, 2])
        new_ind_options = all_ind_options[all_ind_options != inds[i]]
        inds[i] = np.random.choice(new_ind_options) 

# define a nominal vector from population 2 
a2 = 50*np.ones(n)  # low abundance
a2[inds==1] = 200   # medium abundance 
a2[inds==2] = 10000 # High abundance  

In [4]:
# confirm that p*n species are different 
p*n == sum(a1!=a2)

True

In [5]:
# now simulate populations by adding noise to u1 and u2, m1 and m2 times 
# a random scale is included so that measurements reflect abundance counts between samples

# simulate population 1 
X1 = np.zeros([n, m1])
for j in range(m1):    # for each subject
    A_j = np.zeros(n)
    for i in range(n): # for each taxon
        # mu_ij generated from gamma distribution
        mu_ij = gamma.rvs(a1[i], 1)
        # A_ij generated from poisson distribution
        A_j[i] = poisson.rvs(mu_ij)
    # generate abundance at specimen level for j_th subject
    # by scaling A_j by factor c_j sampled from uniform
    scale = 1/np.random.uniform(100, 200)
    # only keep integer part of each taxa 
    Y_j = np.array(scale*A_j, np.int) 
    X1[:, j] = Y_j
    
# define lower and upper bounds for sampling from uniform
l = [100, 200, 10000]
u = [150, 400, 15000]
    
# simulate population 2 
X2 = np.zeros([n, m1])
for j in range(m1):    # for each subject
    A_j = np.zeros(n)
    for i in range(n): # for each taxon
        # mu_ij sampled from gamma distribution
        mu_ij = gamma.rvs(a2[i], 1) 
        # u_ij sampled from uniform 
        u_ij = np.random.uniform(l[inds[i]], u[inds[i]])
        # A_ij sampled from poisson distribution
        A_j[i] = poisson.rvs(mu_ij + u_ij)
    # generate abundance at specimen level for j_th subject
    # by scaling A_j by factor c_j sampled from uniform
    scale = 1/np.random.uniform(100, 200)
    # only keep integer part of each taxa 
    Y_j = np.array(scale*A_j, np.int) 
    X2[:, j] = Y_j

In [6]:
# first we'll see what happens when the taxa are not compositional 

# perform t-test to determine whether taxa are differentially abundant 
positives = np.zeros(n)
for i in range(n):
    # for each taxa, perform t-test 
    stat, pvalue = tt(X1[i, :], X2[i, :])
    if pvalue < .05:
        positives[i] = 1

In [7]:
# calculate the number of false positives (instances where positives == 1 but dff_inds != 1)
inds_positive = positives == 1

true_positives = np.sum(dff_inds[inds_positive])
false_positives = np.sum(dff_inds[inds_positive] == 0) 
false_negatives = np.sum(positives[dff_inds==1]==0)

In [8]:
acc = sum(positives == dff_inds) / n
FDR = false_positives / (false_positives + true_positives)

print("Accuracy: {:.3f}".format(acc))
print("False discovery rate: {:.3f}".format(FDR))

Accuracy: 0.199
False discovery rate: 0.801


Now re-do the analysis after normalizing the data 

In [9]:
X1n = X1 / np.sum(X1, 0)
X2n = X2 / np.sum(X2, 0)

# first we'll see what happens when the taxa are not compositional 

# perform t-test to determine whether taxa are differentially abundant 
positives = np.zeros(n)
for i in range(n):
    # for each taxa, perform t-test 
    stat, pvalue = tt(X1n[i, :], X2n[i, :])
    if pvalue < .05:
        positives[i] = 1
        
# calculate the number of false positives (instances where positives == 1 but dff_inds != 1)
inds_positive = positives == 1

true_positives = np.sum(dff_inds[inds_positive])
false_positives = np.sum(dff_inds[inds_positive] == 0) 
false_negatives = np.sum(positives[dff_inds==1]==0)

In [10]:
acc = sum(positives == dff_inds) / n
FDR = false_positives / (false_positives + true_positives)

print("Accuracy: {:.3f}".format(acc))
print("False discovery rate: {:.3f}".format(FDR))

Accuracy: 0.652
False discovery rate: 0.635


Now try taking a log transform of the data and see what happens

In [11]:
X1l = np.log(X1n + .001)
X2l = np.log(X2n + .001) 

# first we'll see what happens when the taxa are not compositional 

# perform t-test to determine whether taxa are differentially abundant 
positives = np.zeros(n)
for i in range(n):
    # for each taxa, perform t-test 
    stat, pvalue = tt(X1l[i, :], X2l[i, :])
    if pvalue < .05:
        positives[i] = 1
        
# calculate the number of false positives (instances where positives == 1 but dff_inds != 1)
inds_positive = positives == 1

true_positives = np.sum(dff_inds[inds_positive])
false_positives = np.sum(dff_inds[inds_positive] == 0) 
false_negatives = np.sum(positives[dff_inds==1]==0)

In [12]:
acc = sum(positives == dff_inds) / n
FDR = false_positives / (false_positives + true_positives)

print("Accuracy: {:.3f}".format(acc))
print("False discovery rate: {:.3f}".format(FDR))

Accuracy: 0.653
False discovery rate: 0.634
